In [1]:
import pandas as pd
import requests, json

# 設定日期和股票代號
date = '20230901'
stock_no = '2330'

# 構建正確的請求 URL，確保 date 參數正確傳遞
url = requests.get(f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date}&stockNo={stock_no}')

# 解析 JSON 回應
content = url.json()  # 使用 requests 自帶的 .json() 方法

# 提取所需的數據
stock_data = content['data']
col_name = content['fields']

# 將數據轉換為 DataFrame
df = pd.DataFrame(data=stock_data, columns=col_name)


# print(df)

output_data={
    'field': col_name,
    'data' : stock_data
}


file_name = f'stock_data_{stock_no}_{date}.csv'
df.to_csv(file_name,index=False,encoding='utf-8-sig')

print(f'Data saved to {file_name}')

Data saved to stock_data_2330_20230901.csv


In [ ]:
import sqlite3

sql = '''
        CREATE TABLE IF NOT EXIST record (
	日期 TEXT NOT NULL,
	成交筆數 INTEGER,
	開盤價 NUMERIC,
	最高價 NUMERIC,
	最低價 NUMERIC,
	收盤價 NUMERIC
    );
    '''



In [ ]:
import sqlite3
conn = sqlite3.connect("stock_data.db")
with conn:
    sql = '''
            CREATE TABLE IF NOT EXISTS record (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        日期 TEXT UNIQUE,
        開盤價 NUMERIC,
        最高價 NUMERIC,
        最低價 NUMERIC,
        成交筆數 INTEGER,
        收盤價 INTEGER NOT NULL
    );
    '''

    cursor = conn.cursor()
    cursor.execute(sql)




In [ ]:
! pip install yfinance

In [ ]:
import yfinance as yf
import sqlite3
import pandas as pd
import numpy as np

start = '2020-01-01'
end = '2024-11-05'
symbol = '2330.TW'

# 下載資料
data = yf.download(symbol, start=start, end=end)

# 連接到 SQLite 資料庫
conn = sqlite3.connect('stockdb.db')
cursor = conn.cursor()

# 建立表格
sql = '''
    CREATE TABLE IF NOT EXISTS stock_prices (
        symbol TEXT,
        date DATE UNIQUE,  
        open REAL,
        high REAL,
        low REAL,
        adj_close REAL,
        volume INTEGER,
        close REAL
    )
'''
cursor.execute(sql)

# 插入資料，確保格式正確
for index, row in data.iterrows():
    # 將 index 轉換為字串格式日期
    date_str = index.strftime('%Y-%m-%d') if isinstance(index, pd.Timestamp) else str(index)
    
    # 檢查每個欄位是否是 NaN，並轉換為 None
    open_price = row['Open'] if pd.notna(row['Open']) else None
    high_price = row['High'] if pd.notna(row['High']) else None
    low_price = row['Low'] if pd.notna(row['Low']) else None
    adj_close_price = row['Adj Close'] if pd.notna(row['Adj Close']) else None
    volume = int(row['Volume']) if pd.notna(row['Volume']) else None
    close_price = row['Close'] if pd.notna(row['Close']) else None

    # 插入資料到 SQLite 資料庫
    cursor.execute('''
        INSERT OR IGNORE INTO stock_prices (symbol, date, open, high, low, adj_close, volume, close)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (symbol, date_str, open_price, high_price, low_price, adj_close_price, volume, close_price))

# 提交並關閉連線
conn.commit()
cursor.close()
conn.close()

In [13]:
import yfinance as yf

# 設定下載的股票代號與日期範圍
symbol = '2330.TW'
start = '2023-01-01'
end = '2024-01-01'

# 下載資料
data = yf.download(symbol, start=start, end=end)

# 顯示資料前幾行
print(data.head())

[*********************100%***********************]  1 of 1 completed

Price                       Adj Close   Close    High     Low    Open  \
Ticker                        2330.TW 2330.TW 2330.TW 2330.TW 2330.TW   
Date                                                                    
2023-01-03 00:00:00+00:00  437.941528   453.0   453.5   443.0   446.0   
2023-01-04 00:00:00+00:00  434.557861   449.5   455.0   448.5   449.5   
2023-01-05 00:00:00+00:00  443.258698   458.5   459.5   455.0   459.0   
2023-01-06 00:00:00+00:00  443.258698   458.5   459.5   455.0   455.0   
2023-01-09 00:00:00+00:00  465.010773   481.0   481.0   467.5   468.0   

Price                        Volume  
Ticker                      2330.TW  
Date                                 
2023-01-03 00:00:00+00:00  14885824  
2023-01-04 00:00:00+00:00  19188422  
2023-01-05 00:00:00+00:00  23549581  
2023-01-06 00:00:00+00:00  20886011  
2023-01-09 00:00:00+00:00  46666263  


In [ ]:
import yfinance as yf
import sqlite3
import pandas as pd

# 設定下載的股票代號與日期範圍
symbol = '2330.TW'
start = '2023-01-01'
end = '2024-01-01'

# 下載資料
data = yf.download(symbol, start=start, end=end)

# 連接到 SQLite 資料庫（如果資料庫不存在，會自動創建）
conn = sqlite3.connect('stocks_data.db')

# 創建資料表
sql = '''
    CREATE TABLE IF NOT EXISTS NewTable (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        Date TEXT NOT NULL UNIQUE,
        "Open" NUMERIC,
        High NUMERIC,
        Low NUMERIC,
        "Adj Close" NUMERIC,
        Volume NUMERIC,
        "Close" NUMERIC
    );
'''

# 執行 SQL 指令來創建資料表
conn.execute(sql)

    
    
   
# 提交更改並關閉連接
conn.commit()
cursor.close()
conn.close()

[*********************100%***********************]  1 of 1 completed

2023-01-03 00:00:00+00:00
Price      Ticker 
Adj Close  2330.TW    4.379415e+02
Close      2330.TW    4.530000e+02
High       2330.TW    4.535000e+02
Low        2330.TW    4.430000e+02
Open       2330.TW    4.460000e+02
Volume     2330.TW    1.488582e+07
Name: 2023-01-03 00:00:00+00:00, dtype: float64
2023-01-04 00:00:00+00:00
Price      Ticker 
Adj Close  2330.TW    4.345579e+02
Close      2330.TW    4.495000e+02
High       2330.TW    4.550000e+02
Low        2330.TW    4.485000e+02
Open       2330.TW    4.495000e+02
Volume     2330.TW    1.918842e+07
Name: 2023-01-04 00:00:00+00:00, dtype: float64
2023-01-05 00:00:00+00:00
Price      Ticker 
Adj Close  2330.TW    4.432587e+02
Close      2330.TW    4.585000e+02
High       2330.TW    4.595000e+02
Low        2330.TW    4.550000e+02
Open       2330.TW    4.590000e+02
Volume     2330.TW    2.354958e+07
Name: 2023-01-05 00:00:00+00:00, dtype: float64
2023-01-06 00:00:00+00:00
Price      Ticker 
Adj Close  2330.TW    4.432587e+02
Close      

In [12]:


conn = sqlite3.connect('stock_data.db')
cursor = conn.cursor()
# 遍歷 DataFrame 中的每一行資料
for index, row in data.iterrows():
    # 強制將索引轉換為 datetime 並格式化為字串
    date_str = pd.to_datetime(index).strftime('%Y-%m-%d')
     # 確保所有數值型資料都能正確處理
    insertSQL = """
        INSERT INTO NewTable (Date, "Open", High, Low, "Adj Close", Volume, "Close")
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """
cursor.execute(insertSQL, (date_str, row['Open'], row['High'], row['Low'], row['Adj Close'], row['Volume'], row['Close']))
conn.commit()
cursor.close()
conn.close()


InterfaceError: Error binding parameter 1 - probably unsupported type.

In [ ]:
import yfinance as yf
import sqlite3
import pandas as pd

# 設定下載的股票代號與日期範圍
symbol = '2330.TW'
start = '2023-01-01'
end = '2024-01-01'

# 下載資料
data = yf.download(symbol, start=start, end=end)
yf.download()
# 連接到 SQLite 資料庫（如果資料庫不存在，會自動創建）
conn = sqlite3.connect('stock_data.db')
cursor = conn.cursor()

# 創建資料表
sql = '''
    CREATE TABLE IF NOT EXISTS NewTable (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        Date TEXT NOT NULL UNIQUE,
        "Open" NUMERIC,
        High NUMERIC,
        Low NUMERIC,
        "Adj Close" NUMERIC,
        Volume NUMERIC,
        "Close" NUMERIC
    );
'''
# 執行 SQL 指令來創建資料表
conn.execute(sql)
conn.commit()

# 遍歷 DataFrame 中的每一行資料並插入資料庫
for index, row in data.iterrows():
    # 強制將索引轉換為 datetime 並格式化為字串
    date_str = pd.to_datetime(index).strftime('%Y-%m-%d')
    
    # 插入資料
    insertSQL = """
        INSERT OR IGNORE INTO NewTable (Date, "Open", High, Low, "Adj Close", Volume, "Close")
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """
    cursor.execute(insertSQL, (date_str, row['Open'], row['High'], row['Low'], row['Adj Close'], row['Volume'], row['Close']))

# 提交更改並關閉連接
conn.commit()
cursor.close()
conn.close()


[*********************100%***********************]  1 of 1 completed


InterfaceError: Error binding parameter 1 - probably unsupported type.

In [11]:
import yfinance as yf
import sqlite3
import pandas as pd

# 設定下載的股票代號與日期範圍
symbol = '2330.TW'
start = '2023-01-01'
end = '2024-01-01'

# 下載資料
data = yf.download(symbol, start=start, end=end)

# 連接到 SQLite 資料庫（如果資料庫不存在，會自動創建）
conn = sqlite3.connect('stock_data.db')
cursor = conn.cursor()


# 遍歷 DataFrame 中的每一行資料並插入資料庫
for index, row in data.iterrows():
    print(f"Date: {index}\nOpen: {row['Open']}\nHigh: {row['High']}\nLow: {row['Low']}\nAdj close={row['Adj Close']}\nVolume: {row['Volume']}\nClose: {row['Close']}\n")
    
    # 強制將索引轉換為 datetime 並格式化為字串
    
    # 插入資料
    insertSQL = """
        INSERT OR IGNORE INTO NewTable (Date, "Open", High, Low, "Adj Close", Volume ,"Close")
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """
    cursor.execute(insertSQL, (index, row['Open'], row['High'], row['Low'], row['Adj Close'], row['Volume'] ,row['Close']))

# 提交更改並關閉連接
conn.commit()
cursor.close()
conn.close()

[*********************100%***********************]  1 of 1 completed

Date: 2023-01-03 00:00:00+00:00
Open: Ticker
2330.TW    446.0
Name: 2023-01-03 00:00:00+00:00, dtype: float64
High: Ticker
2330.TW    453.5
Name: 2023-01-03 00:00:00+00:00, dtype: float64
Low: Ticker
2330.TW    443.0
Name: 2023-01-03 00:00:00+00:00, dtype: float64
Adj close=Ticker
2330.TW    437.941559
Name: 2023-01-03 00:00:00+00:00, dtype: float64
Volume: Ticker
2330.TW    14885824.0
Name: 2023-01-03 00:00:00+00:00, dtype: float64
Close: Ticker
2330.TW    453.0
Name: 2023-01-03 00:00:00+00:00, dtype: float64



InterfaceError: Error binding parameter 0 - probably unsupported type.

In [21]:
import yfinance as yf
import sqlite3
import pandas as pd

# 設定下載的股票代號與日期範圍
symbol = '2330.TW'
start = '2020-01-01'
end = '2024-11-05'

# 下載資料
data = yf.download(symbol, start=start, end=end)

conn = sqlite3.connect('stock_data.db')
cursor = conn.cursor()
# 遍歷 DataFrame 中的每一行資料並插入資料庫
for index, row in data.iterrows():
    date_str = index.strftime('%Y-%m-%d')
    
    # 從 Series 中提取單一值
    open_price = row['Open'].iloc[0] if pd.notnull(row['Open'].iloc[0]) else None
    high = row['High'].iloc[0] if pd.notnull(row['High'].iloc[0]) else None
    low = row['Low'].iloc[0] if pd.notnull(row['Low'].iloc[0]) else None
    adj_close = row['Adj Close'].iloc[0] if pd.notnull(row['Adj Close'].iloc[0]) else None
    volume = row['Volume'].iloc[0] if pd.notnull(row['Volume'].iloc[0]) else None
    close = row['Close'].iloc[0] if pd.notnull(row['Close'].iloc[0]) else None

    # 插入資料
    insertSQL = """
        INSERT OR IGNORE INTO NewTable (Date, "Open", High, Low, "Adj Close", Volume, "Close", Tickers)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """
    cursor.execute(insertSQL, (date_str, open_price, high, low, adj_close, volume, close, symbol))

# 提交更改並關閉連接
conn.commit()
cursor.close()
conn.close()

[*********************100%***********************]  1 of 1 completed


In [22]:
import yfinance as yf
import sqlite3
import pandas as pd

# 設定下載的股票代號與日期範圍
symbol = '2330.TW'
start = '2023-01-01'
end = '2024-01-01'

# 下載資料
data = yf.download(symbol, start=start, end=end)


# 遍歷 DataFrame 中的每一行資料並插入資料庫
for index, row in data.iterrows():
    date_str = index.strftime('%Y-%m-%d')
    
    # 從 Series 中提取單一值
    open_price = row['Open'].iloc[0] if pd.notnull(row['Open'].iloc[0]) else None
    high = row['High'].iloc[0] if pd.notnull(row['High'].iloc[0]) else None
    low = row['Low'].iloc[0] if pd.notnull(row['Low'].iloc[0]) else None
    adj_close = row['Adj Close'].iloc[0] if pd.notnull(row['Adj Close'].iloc[0]) else None
    volume = row['Volume'].iloc[0] if pd.notnull(row['Volume'].iloc[0]) else None
    close = row['Close'].iloc[0] if pd.notnull(row['Close'].iloc[0]) else None

print(row['Open'])
print("===========")
print(row['Open'].iloc[0])
print("===========")
print(row['Open'].iloc[1])
print("===========")
print(len(row['Open']))
print("===========")
for value in row['Open']:
    print(value)



[*********************100%***********************]  1 of 1 completed


Ticker
2330.TW    589.0
Name: 2023-12-29 00:00:00+00:00, dtype: float64
589.0


IndexError: single positional indexer is out-of-bounds

In [3]:
import sqlite3

conn = sqlite3.connect('check1.db')
cursor = conn.cursor()

# 建立資料表，如果尚未存在
create_table_sql = """
CREATE TABLE IF NOT EXISTS NewTable (
    Date TEXT PRIMARY KEY,
    "Open" NUMERIC,
    High NUMERIC,
    Low NUMERIC,
    "Adj Close" NUMERICL,
    Volume NUMERIC,
    "Close" NUMERIC,
    Tickers TEXT
);
"""
cursor.execute(create_table_sql)

cursor.close()
conn.close()

In [4]:
import sqlite3
conn = sqlite3.connect('check_data.db')
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='NewTable';")
result = cursor.fetchone()
if result:
    print("資料表已存在")
else:
    print("資料表不存在")

cursor.close()
conn.close()


資料表已存在
